In [609]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt 
from sklearn.ensemble import IsolationForest
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from xgboost import XGBClassifier

# %matplotlib inline

IMPORTING THE CSV FILES AND SCALING THE DATA ACCORDINGLY

In [610]:
X_full = pd.read_csv('corrected train set.csv', index_col='id')
X_test_full = pd.read_csv('week 1 test.csv', index_col='id')

# num = [cname for cname in X_full.columns if 
#                 X_full[cname].dtype in ['int64', 'float64']]

# SCALING THE DATA FOR EASY COMPUTATION

mm_scaler = MinMaxScaler()

num_d = X_full.select_dtypes(exclude=['object'])

X_full[num_d.columns] = mm_scaler.fit_transform(num_d)


# X_full
# # Remove rows with missing target, separate target from predictors


In [611]:
X_test_full

,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
id,,,,,,,,,,,,,,,,,,,,,
16108,C,145.08,material_7,material_8,5,8,9,7,8,16.863,...,18.218,12.015,15.247,NaN,11.303,16.717,16.768,14.707,17.190,658.222
16109,C,146.34,material_7,material_8,5,8,2,7,1,18.303,...,NaN,NaN,16.215,20.778,9.195,16.047,NaN,15.759,18.393,616.670
16110,C,118.14,material_7,material_8,5,8,1,9,3,16.883,...,16.701,11.181,16.391,20.434,9.365,16.182,15.934,12.487,14.615,433.006
16111,C,119.86,material_7,material_8,5,8,10,9,9,17.686,...,19.437,10.959,16.567,18.430,11.836,15.767,18.049,15.346,15.337,572.642
16112,C,122.66,material_7,material_8,5,8,8,9,9,18.132,...,18.068,10.333,18.631,17.557,11.959,16.202,14.098,14.110,15.025,486.221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26565,E,158.95,material_7,material_6,6,9,6,16,4,16.301,...,19.354,NaN,12.177,17.942,10.112,15.795,18.572,16.144,NaN,729.131
26566,E,146.02,material_7,material_6,6,9,10,12,8,17.543,...,19.563,11.242,14.179,20.564,10.234,14.450,14.322,13.146,16.471,853.924
26567,E,115.62,material_7,material_6,6,9,1,10,1,15.670,...,19.279,11.407,16.437,17.476,8.668,15.069,16.599,15.590,14.065,750.364


CREATING THE MODEL USING XGBClassifier. Tried other model but this gave the maximum ROC AUC

In [612]:

# numerical_transformer = SimpleImputer(strategy='constant')

# # Preprocessing for categorical data
# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='constant')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

# # Bundle preprocessing for numerical and categorical data
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numerical_transformer, numerical_cols),
#         ('cat', categorical_transformer, categorical_cols)
#     ])

# # Define model
# model1 = RandomForestClassifier(n_estimators=100, random_state=0)

model2 = XGBClassifier(n_estimators=10000, learning_rate=0.05, n_jobs=4)

# model = LogisticRegression(random_state=1, max_iter=10000)# Preprocessing for numerical data


USING THE TRAIN TEST SPLIT TO SPLIT THE DATAFRAME FOR TRAINING THE MODEL. THEN MODIFYING THE TRAINING FOR EFFICIENT MODELLING

In [613]:
X_full.dropna(axis=0, subset=['failure'], inplace=True)

y = X_full.failure

X_full.drop(['failure'], axis=1, inplace=True)



# # Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# # "Cardinality" means the number of unique values in a column
# # Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# # Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# # Keep selected columns only
my_cols = categorical_cols + numerical_cols

X_train = X_train_full[my_cols].copy()

X_valid = X_valid_full[my_cols].copy()

X_test = X_test_full[my_cols].copy()


#ENCODING THE CATEGORICAL VARIABLES TO NUMERICAL VARIABLES
X_train2 = pd.get_dummies(X_train)

X_valid2 = pd.get_dummies(X_valid)

X_test2 = pd.get_dummies(X_test)

X_train2, X_valid2 = X_train2.align(X_valid, join='left', axis=1)

X_train2, X_test2 = X_train2.align(X_test, join='left', axis=1)
                                   
          
X_train2.head()


,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,...,measurement_15,measurement_16,measurement_17,product_code_A,product_code_B,product_code_C,attribute_0_material_5,attribute_0_material_7,attribute_1_material_5,attribute_1_material_8
id,,,,,,,,,,,,,,,,,,,,,
5527,0.326765,0.75,1.0,0.310345,0.259259,0.619048,0.430089,0.266281,0.602224,0.284356,...,0.351219,0.364931,0.264598,0,1,0,1,0,1,0
14174,0.203998,0.00,1.0,0.517241,0.111111,0.047619,0.435135,0.696555,0.416061,0.462706,...,0.695861,0.611352,0.504098,0,0,1,0,1,0,1
1530,0.263028,1.00,0.0,0.137931,0.148148,0.285714,0.621830,0.598395,0.571001,0.434191,...,0.462865,NaN,0.489854,1,0,0,0,1,0,1
11008,0.238049,0.00,1.0,0.103448,0.296296,0.380952,0.700571,0.500826,0.719098,0.306931,...,0.647417,0.634996,0.780377,0,0,1,0,1,0,1
9306,0.144003,0.75,1.0,0.103448,0.407407,0.190476,0.571504,0.484191,0.563623,0.700726,...,NaN,0.408821,0.538659,0,1,0,1,0,1,0


In [614]:
X_test2.head()

,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,...,measurement_15,measurement_16,measurement_17,product_code_A,product_code_B,product_code_C,attribute_0_material_5,attribute_0_material_7,attribute_1_material_5,attribute_1_material_8
id,,,,,,,,,,,,,,,,,,,,,
16108,145.08,5,8,9,7,8,16.863,10.156,18.038,16.607,...,14.707,17.190,658.222,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16109,146.34,5,8,2,7,1,18.303,12.670,17.570,15.942,...,15.759,18.393,616.670,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16110,118.14,5,8,1,9,3,16.883,12.024,16.738,16.299,...,12.487,14.615,433.006,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16111,119.86,5,8,10,9,9,17.686,10.697,16.481,16.394,...,15.346,15.337,572.642,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16112,122.66,5,8,8,9,9,18.132,13.269,NaN,16.503,...,14.110,15.025,486.221,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [615]:
X_valid2.head()

,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,...,measurement_15,measurement_16,measurement_17,product_code_A,product_code_B,product_code_C,attribute_0_material_5,attribute_0_material_7,attribute_1_material_5,attribute_1_material_8
id,,,,,,,,,,,,,,,,,,,,,
2459,0.283442,1.00,0.0,0.517241,0.185185,0.238095,0.689284,0.456111,0.498610,0.691749,...,0.335900,0.325163,0.557192,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9491,0.374086,0.75,1.0,0.275862,0.222222,0.285714,0.714646,0.362907,0.764970,0.593531,...,0.606352,0.544179,0.444806,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13833,0.294925,0.00,1.0,0.172414,0.222222,0.142857,0.341389,0.590845,0.422263,0.519736,...,NaN,NaN,0.380007,NaN,NaN,NaN,NaN,NaN,NaN,NaN
578,0.135214,1.00,0.0,0.413793,0.074074,0.285714,0.415748,0.582822,0.478614,0.430891,...,0.476019,0.341432,0.366234,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7637,0.479189,0.75,1.0,0.275862,0.296296,0.333333,0.455451,0.125177,0.482036,0.148911,...,0.325634,0.386840,0.290812,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [616]:
# my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
#                               ('model', model1)
#                              ])

# # Preprocessing of training data, fit model 
# my_pipeline.fit(X_train, y_train)

# # Preprocessing of validation data, get predictions
# # preds = my_pipeline.predict(X_valid)
# # print(preds)

# y_pred_proba = my_pipeline.predict_proba(X_valid)[::,1]
# print(y_pred_proba)

# #calculate AUC of model
# auc = metrics.roc_auc_score(y_valid, y_pred_proba)
# print(auc)
# # Evaluate the model
# # score = mean_absolute_error(y_valid, preds)
# # print('MAE:', score)

FITING THE MODEL AND THEN CALCULATING PREDICT_PROBA TO CALCULATE THE VALUE

In [617]:
X_train.head()

#FIT THE TRAIN IN THE MODEL
model2.fit(X_train2, y_train, early_stopping_rounds=10, 
             eval_set=[(X_valid2, y_valid)], 
             verbose=False)

y_pred_proba = model2.predict_proba(X_test2)[::,1] 

print(y_pred_proba)

y_pred_proba2 = model2.predict_proba(X_valid2)[::,1] 

auc2 = metrics.roc_auc_score(y_valid, y_pred_proba2) 
print(auc2)
X_test_full['product_code']
X_test_full.insert(2, "Failiure Probability", y_pred_proba, True)
# X_test_full.head()
Result=X_test_full
Result.head()

Result.to_csv('RESULT.csv', header=True, index=True)
# final_result = pd.DataFrame(y_pred_proba, columns=['Failiures'],index=X_test2['id'])

/Users/akshat/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/akshat/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[18:10:56] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0.5249186  0.49067563 0.5420545  ... 0.5420545  0.5203819  0.5420545 ]
0.5620443341975085
